# 준비작업

### import

In [3]:
import pyautogui as au
import pyperclip
import time
import pandas as pd
import re
import pickle
from os.path import join
import os
import numpy as np
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
import random
# 정렬순서 : 확정일자(내림), 법원(올림), 사건번호(내림)

### 함수, 전역변수

In [ ]:
##################################
path_excel = r"D:\0.전산\관리팀업무\전자소송\221215\집행문신청_221209~221215.xlsx"
##################################
bank = "신한은행"
driver, wait = None, None

# pause
def lwt() : time.sleep(2.5)
def mwt() : time.sleep(1)
def swt() : time.sleep(random.uniform(0.1,0.4))

def con_scourt() :
    service = Service(executable_path="./파일/chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    driver.get("https://ecfs.scourt.go.kr/ecf/index.jsp")
    wait = WebDriverWait(driver, 30) 
    wait.until(EC.element_to_be_clickable((By.ID, 'userId1')))
    mwt()
    return driver, wait

def certify() :
    elem = wait.until(EC.element_to_be_clickable((By.ID, "xwup_certselect_tek_input1")))
    mwt()
    elem.send_keys("help8700765!")
    swt()
    driver.find_element(By.ID, 'xwup_OkButton').click() # 인증서 확인
    mwt()

def position_new_win():
    position = driver.get_window_position()
    x = position.get('x')
    y = position.get('y')
    return x, y

def login() : 
    # 로그인(id) # 셀레니움 안 돼서.. 
    x, y = position_new_win()
    au.click(x+100,y+305)
    time.sleep(0.2)
    au.write("help")
    au.press("num2")
    au.press("num2")
    au.write("55")
    driver.find_element(By.ID, "userId1").send_keys(Keys.ENTER)
    # 로그인(인증서)
    certify()
    lwt()

def apply_or_print(AorP : int) :
    "apply_or_print : 신청 : 1, 발급 : 2"
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gnb"]/ul/li[4]/a'))).send_keys(Keys.ENTER) # 제증명
    driver.find_element(By.XPATH, '//*[@id="tmenu04"]/div/ul/li['+str(AorP)+']/a').send_keys(Keys.ENTER) # 제증명신청/발급
    swt()

def apply_certificates(docu_kind : int) :
    "docu_kind : 확정증명:0 | 송달증명:1 | 집행문:3"
    apply_or_print(1)
    Select(driver.find_element(By.ID, 'cbxDocCd')).select_by_index(docu_kind) # 확정증명 선택
    swt()

def select_court_event(court, event) :# driver
    # 법원
    correct_court = re.sub("지법", "지방법원", court)
    Select(driver.find_element(By.ID, 'cbxBubCd')).select_by_visible_text(correct_court) # 집행문 선택
    swt()
    # 사건번호 선택
    event = re.search("(\d{4,4})([가-힣]+)(\d+)", event)
    year = event[1]
    sign = event[2]
    sn = event[3]
    Select(driver.find_element(By.XPATH, '//select[@title="사건연도"]')).select_by_value(year) # 연도 선택
    Select(driver.find_element(By.XPATH, '//select[@title="사건구분"]')).select_by_visible_text(sign)  # 사건구분 선택
    driver.find_element(By.XPATH, '//input[contains(@title,"사건일련번호")]').send_keys(sn)  # 사건구분 선택
    swt()

def select_parties_new_win() : ##############
    # 발급당사자 버튼
    wait.until(EC.element_to_be_clickable((By.ID, 'btnSelBalDs'))).click() # 발급당사자선택 버튼
    mwt()

    # 새창 전환
    driver.switch_to.window(driver.window_handles[1]) # 창 전환
    # 팝업창 컨트롤(발급당사자)
    driver.find_element(By.ID, 'allCheck').click() # 당사자 선택
    driver.find_element(By.ID, 'btnReg').click() # 등록
    swt()
    driver.find_element(By.ID, 'selectedAllCheck').click()  # 선택된 당사자 선택
    driver.find_element(By.ID, 'btnSetDs').click() # 확인
    swt()
    # 창 되돌리기
    driver.switch_to.window(driver.window_handles[0]) # 창 전환

def check_document() : #############
    time.sleep(7) # pdf문서 대기
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="pdfReader"]/iframe')))
    
    # 문서 내용 확인 및 스크롤 (안 되네... )
    # driver.switch_to.frame(driver.find_elements(By.XPATH, '//*[@id="pdfReader"]/iframe')) # 문서viewer iframe 전환
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # 문서 끝까지 스크롤 
    # driver.find_element(By.XPATH, '//*[@id="body"]/sd-root/sd-main/sd-toolbar/section[3]/section[2]/div/pu-button/div/pu-icon').click()
    
    # 문서 이상없음 체크 및 확인버튼
    driver.find_element(By.ID, 'guidcheck').click() # 문서이상없음
    driver.find_element(By.ID, 'btnNextPage').click() # 확인
    mwt()

def send_confirm() :
    # 제출 및 확인창
    wait.until(EC.element_to_be_clickable((By.ID, 'btnSendJchul'))).click() # 제출 버튼
    swt()
    Alert(driver).accept() # 확인alert
    mwt()

def fix_delivery(which:int, v) :
    "which : 확정 - 0, 송달 - 1 \t v : df.iterrows()[1]"
    apply_certificates(which) # 제증명신청, 제증명 종류
    select_court_event(v.법원, v.사건번호) # 법원 사건번호
    driver.find_element(By.ID, 'btnConfirm').click() # 확인 버튼
    mwt()
    wait.until(EC.element_to_be_clickable((By.ID, "sinGbnCd2"))).click() # 당사자별
    swt()
    select_parties_new_win() # 발급당사자선택 ~ 새창 컨트롤
    driver.find_element(By.ID, "btnComplete").click() # 작성완료
    mwt()
    check_document() # 문서확인
    send_confirm() # 제출 및 comfirm
    certify() # 인증서 
    lwt() # 제출 후 인증은 lwt()추가!
    wait.until(EC.element_to_be_clickable((By.ID, 'btnSendList'))).click() # 완료버튼
    swt()
    # df수정
    if which == 0 : v.확정 = "신청"
    elif which == 1 : v.송달 = "신청"

def confirm_print() :   # 게시물에 맞는 발급버튼 누르기
    while(True) :
        win_print = au.getWindowsWithTitle("인쇄")
        if win_print == [] :
            time.sleep(3)
        else :
            break
    win_print[0].activate()
    time.sleep(0.2)
    au.press('enter')

def execution_print() :
    try :
        driver.find_element(By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]/a').click() # 확정증명의 발급 버튼 누르기
    except :
        return driver.find_element(By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]').text
    


    return "발급"

def docu_print(kind_docu:str) :
    # 확정증명 발급. a태그가 없는 경우 td의 text값을 리턴(기발급건은 '발 급'으로 정상적인 '발급'과 구별된다.)
    try :
        driver.find_element(By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]/a').click() # 확정증명의 발급 버튼 누르기
    except :
        return driver.find_element(By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]').text
    mwt()
    Alert(driver).accept()
    time.sleep(6)
    # 인쇄 확인 버튼
    confirm_print()
    # 송달증명 발급
    time.sleep(5)
    Alert(driver).accept() # 컨펌창 확인
    mwt()
    return "발급"

### driver 열기

In [9]:
driver, wait = con_scourt()
login() # 로그인 : 수동으로 하자. 인증서 되는 거 확인

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@title="송달증명"]/parent::td/following-sibling::td[4]/a"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0078F243]
	(No symbol) [0x00717FD1]
	(No symbol) [0x0060D04D]
	(No symbol) [0x0063C0B0]
	(No symbol) [0x0063C22B]
	(No symbol) [0x0066E612]
	(No symbol) [0x006585D4]
	(No symbol) [0x0066C9EB]
	(No symbol) [0x00658386]
	(No symbol) [0x0063163C]
	(No symbol) [0x0063269D]
	GetHandleVerifier [0x00A29A22+2655074]
	GetHandleVerifier [0x00A1CA24+2601828]
	GetHandleVerifier [0x00838C0A+619850]
	GetHandleVerifier [0x00837830+614768]
	(No symbol) [0x007205FC]
	(No symbol) [0x00725968]
	(No symbol) [0x00725A55]
	(No symbol) [0x0073051B]
	BaseThreadInitThunk [0x765900F9+25]
	RtlGetAppContainerNamedObjectPath [0x77DE7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77DE7B8E+238]


### 엑셀 열기

In [7]:
### 원본이 필요할 땐 원본을 다른 이름으로 복사해둬!
df_origin = pd.read_excel(path_excel).fillna("")

In [80]:
df = df_origin.copy()
df

,번호,법원,사건번호,재판부,확정일자,원고,피고,집행문,확정,송달,비고,집행비용,가상계좌번호,채무자키,계좌키
0,1,부산지법,2022가소605974,민사제28단독,2022.12.10,주식회사 솔림헬프자산관리대부,정순학,신청,,,,,56216072432650.0,,
1,2,부산지법,2022가소605394,민사제28단독,2022.12.10,주식회사 솔림헬프자산관리대부,주계송,신청,,,,,56216072443874.0,,
2,3,부산지법,2022가소598168,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,차남권,신청,,,,,56216072444699.0,,
3,4,부산지법,2022가소598182,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,이동건,신청,,,,,56216072446158.0,,
4,5,부산지법,2022가소598199,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,정남희,신청,,,,,56216072447081.0,,
5,6,부산지법,2022가소598205,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,김영훈,신청,,,,,56216072466407.0,,
6,7,부산지법,2022가소598212,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,정민호,신청,,,,,56216072467842.0,,
7,8,부산지법,2022가소598229,민사제27단독,2022.12.10,주식회사 솔림헬프자산관리대부,천태기,신청,,,,,56216072468258.0,,
8,9,부산지법,2022가소598243,민사제27단독,2022.12.10,주식회사 솔림헬프자산관리대부,조종곤,신청,,,,,56216072468347.0,,
9,10,부산지법,2022가소598281,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,유예슬,신청,,,,,56216072468420.0,,


# 집행문

In [44]:
# 대법원 조회
cnt_success, cnt_fail = 0, 0
for i, v in df.iterrows() :
    if v.집행문 == "" :
        try :
            # 제증명 신청(1), 집행문(4)
            apply_certificates(3)
            # 법원 및 사건번호 및 확인
            select_court_event(v.법원, v.사건번호)
            driver.find_element(By.ID, 'btnConfirm').click() # confirm
            mwt()

            # 발급당사자
            select_parties_new_win() # 발급당사자 새창

            # 동의 및 작성완료
            driver.find_element(By.ID, 'chkAgrElcSo').click() # 동의 체크박스
            driver.find_element(By.ID, 'btnComplete').click() # 작성완료
            mwt()
            Alert(driver).accept() # confirm
            
            # 문서이상없음 및 확인
            check_document() 

            # 은행선택 및 납부
            Select(driver.find_element(By.ID, 'vtBankCd')).select_by_visible_text(bank) # 은행 선택
            driver.find_element(By.ID, 'btnElcPay').click() # 납부 버튼
            swt()
            Alert(driver).accept() # confirm
            mwt()
            
            # 제출 및 confirm 확인
            send_confirm()
                
            # 인증서
            certify()
            lwt()

            # 계좌정보 담기 및 완료버튼
            account = driver.find_element(By.ID, 'acno').text # 계좌정보 담기
            wait.until(EC.element_to_be_clickable((By.ID, 'btnSendList'))).click() # 완료버튼
            swt()

            # df 수정 : 계좌번호 및 집행문 상태
            df.loc[i, "가상계좌번호"] = account
            df.loc[i, "집행문"] = "신청"
            cnt_success += 1
        except : 
            cnt_fail += 1
            continue

# 드라이버 종료 및 엑셀 저장
print(f"성공 {cnt_success}건, 실패 {cnt_fail}건")
driver.quit()
df.to_excel(path_excel, index=False)
        

,번호,법원,사건번호,재판부,확정일자,원고,피고,집행문,송달,확정,비고,집행비용,가상계좌번호
0,1,부산지법,2022가소605974,민사제28단독,2022.12.10,주식회사 솔림헬프자산관리대부,정순학,신청,nan,nan,nan,nan,56216072432650.0
1,2,부산지법,2022가소605394,민사제28단독,2022.12.10,주식회사 솔림헬프자산관리대부,주계송,신청,nan,nan,nan,nan,56216072443874.0
2,3,부산지법,2022가소598168,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,차남권,신청,nan,nan,nan,nan,56216072444699.0
3,4,부산지법,2022가소598182,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,이동건,신청,nan,nan,nan,nan,56216072446158.0
4,5,부산지법,2022가소598199,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,정남희,신청,nan,nan,nan,nan,56216072446158.0
5,6,부산지법,2022가소598205,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,김영훈,신청,nan,nan,nan,nan,56216072466407.0
6,7,부산지법,2022가소598212,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,정민호,신청,nan,nan,nan,nan,56216072467842.0
7,8,부산지법,2022가소598229,민사제27단독,2022.12.10,주식회사 솔림헬프자산관리대부,천태기,신청,nan,nan,nan,nan,56216072468258
8,9,부산지법,2022가소598243,민사제27단독,2022.12.10,주식회사 솔림헬프자산관리대부,조종곤,신청,nan,nan,nan,nan,56216072468347
9,10,부산지법,2022가소598281,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,유예슬,신청,nan,nan,nan,nan,56216072468420


# 확정증명원/송달증명원

In [138]:
cnt_success_fix, cnt_success_del, cnt_fail = 0, 0
for i, v in df.iterrows() :
    ###################
    if v.번호 < 20 : pass
    else :
    ###################
        try :
            if v.확정 == "" :
                fix_delivery(0, v)
                df.loc[i, "확정"] = "신청" # df수정
                cnt_success_fix += 1

            if v.송달 == "" :
                fix_delivery(1, v)
                df.loc[i, "송달"] = "신청" # df수정
                cnt_success_del += 1
        except :
            cnt_fail += 1

print(f"신청성공[확정증명원 : {cnt_success_fix}건, 송달증명원 : {cnt_success_del}건],    실패 {cnt_fail}건")
# 엑셀 저장
df.to_excel(path_excel, index=False)

# 출력

In [141]:
cnt_success, cnt_success_fix, cnt_success_del, cnt_fail = 0, 0, 0, 0
for i, v in df.iterrows() :
    ###################
    if v.번호 < 20 : pass
    else :
    ###################
        # 1) 발급할 것이 있는 사건번호의 경우 검색한다.
        try :
            if (v.집행문 == "신청") | (v.확정 == "신청") or (v.송달 == "신청") : 
                apply_or_print(2) # 제증명 발급
                driver.find_element(By.ID, "selSa").click() # 사건번호 radio
                swt()
                select_court_event(v.법원, v.사건번호)
                driver.find_element(By.ID, "btnRetrieve").click()
                mwt()

            # 2) 해당하는 것만 발급한다.
            ######################
            if v.집행문 == "신청" :
                df.loc[i, "집행문"] = execution_print("집행문(정본포함)")
            ######################

            if v.확정 == "신청" :
                df.loc[i, "확정"] = docu_print("확정증명")
                
            if v.송달 == "신청" : # 송달도 발급해야 하면 연속 발급
                df.loc[i, "송달"] = docu_print("송달증명")
        except :
            cnt_fail += 1
            continue

print(f"출력성공[집행문:{cnt_success}건, 확정:{cnt_success_fix}건, 송달:{cnt_success_del}건,   실패 : {cnt_fail}건")
# 드라이버 종료 및 엑셀 저장
driver.quit()
df.to_excel(path_excel, index=False)

# 후속업무 : 법조치, 메모, 스캔 등

### df_law 불러오기

In [37]:
# cols = 채무자키, 매각사구분, 법조치구분, 관할법원, 사건번호, 청구금액법원, 관련사건번호, 매각사
path_xlsx = r'D:\0.전산\관리팀업무\전자소송' 
path_pkl = path_xlsx
##########################################
file_name = "법조치조회새창_참조용_20221207_1309"
###########################################

# # 자료 업데이트
# df_law = pd.read_excel(join(path_xlsx, file_name + ".xlsx"))
# df_law.to_pickle(join(path_pkl, file_name + ".pkl"))

# pkl 읽기
df_law = pd.read_pickle(join(path_pkl, file_name + ".pkl"))
df_law.head(1)

,채무자키,매각사구분,법조치키,법조치구분,법조치세부,관할법원,사건번호,비고,계좌키,접수일,...,법조치기간,관련사건법원,관련사건번호,매각사,타법조치키,원채권사,법조치메모,제소신청일,등록일자,등록유저
0,20407790,DNP-01,20678164,지급명령,지급명령 신청,서울중앙지방법원,2014차전240794,NaN,200919485,2014-10-24,...,NaN,NaN,NaN,디엔피에이엠씨대부㈜,25348.0,외환카드,NaN,NaN,2019-01-02,system


### df에 채무자키 계좌키 제소신청일 추가하기

In [86]:
for i, v in df.iterrows() :
    df_result = df_law[(df_law.관련사건번호== v.사건번호) |(df_law.사건번호== v.사건번호)].loc[:,["채무자키", "계좌키", "제소신청일"]].drop_duplicates(subset="채무자키").fillna("")
    df_result["채무자키"].astype(int)
    if len(df_result) == 1 :
        df.loc[i, ["채무자키","계좌키", "제소신청일"]] = np.concatenate(df_result.values).tolist() # 1차원 리스트 이기는 한데 채무자키가 소수점이 되네..

        # df.loc[i, ["채무자키"]] = df_result["채무자키"].values
        # df.loc[i, ["계좌키"]] = df_result["계좌키"].values
        # df.loc[i, ["제소신청일"]] = df_result["제소신청일"].values
df.fillna("", inplace=True)
# 엑셀에 저장
df.to_excel(path_excel, index=False)
df

,번호,법원,사건번호,재판부,확정일자,원고,피고,집행문,확정,송달,비고,집행비용,가상계좌번호,채무자키,계좌키,제소신청일
0,1,부산지법,2022가소605974,민사제28단독,2022.12.10,주식회사 솔림헬프자산관리대부,정순학,신청,,,,,56216072432650.0,20434197,200955198,
1,2,부산지법,2022가소605394,민사제28단독,2022.12.10,주식회사 솔림헬프자산관리대부,주계송,신청,,,,,56216072443874.0,20430777,200951583,2022-10-25
2,3,부산지법,2022가소598168,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,차남권,신청,,,,,56216072444699.0,20433804,200954799,2022-09-30
3,4,부산지법,2022가소598182,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,이동건,신청,,,,,56216072446158.0,20436925,200958011,2022-09-30
4,5,부산지법,2022가소598199,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,정남희,신청,,,,,56216072447081.0,20435023,200956051,2022-09-30
5,6,부산지법,2022가소598205,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,김영훈,신청,,,,,56216072466407.0,20437779,200958881,2022-09-30
6,7,부산지법,2022가소598212,민사 제26단독(소액),2022.12.10,주식회사 솔림헬프자산관리대부,정민호,신청,,,,,56216072467842.0,20431231,200952047,
7,8,부산지법,2022가소598229,민사제27단독,2022.12.10,주식회사 솔림헬프자산관리대부,천태기,신청,,,,,56216072468258.0,20432499,200953450,
8,9,부산지법,2022가소598243,민사제27단독,2022.12.10,주식회사 솔림헬프자산관리대부,조종곤,신청,,,,,56216072468347.0,20495329,201018468,2022-09-30
9,10,부산지법,2022가소598281,민사제24단독,2022.12.10,주식회사 솔림헬프자산관리대부,유예슬,신청,,,,,56216072468420.0,20440084,200961371,2022-09-30


## 스캔파일

In [149]:
path = r"D:\scan\새 폴더"
p_e = re.compile("\d{4}[가-힣]{1,3}\d+")
df_result = pd.DataFrame(columns=["채무자키", "계좌키", "사건번호", "계좌키", "제소신청일"])
for f in os.listdir(path) :
    stem, ext = os.path.splitext(f)
    e_name = p_e.search(stem).group()
    # drop_duplicates(subset=["채무자키"], keep='first').
    keys = df_law[(df_law.관련사건번호== e_name) |(df_law.사건번호== e_name)].loc[:,["채무자키", "계좌키", "사건번호", "계좌키", "제소신청일"]]
    df_result = pd.concat([df_result, keys])
    
df_result    

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'D:\\scan\\새 폴더'